In [ ]:
import openai
import gradio as gr
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Fetch the OpenAI API key from the environment
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to generate responses
def generate_response(message, history):
    # Format the conversation history for better continuity
    formatted_history = []
    for user, assistant in history:
        formatted_history.append({"role": "user", "content": user})
        formatted_history.append({"role": "assistant", "content": assistant})
    
    formatted_history.append({"role": "user", "content": message})
    
    # Call to OpenAI API
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=formatted_history,
        temperature=0.7,  # Lower temperature for more deterministic responses
        max_tokens=300,   # Adjust token limit as needed
    )

    # Return the assistant's response
    return response.choices[0].message['content']

# Gradio interface setup
def chat_interface():
    with gr.Blocks() as app:
        chatbot = gr.Chatbot(height=300)
        msg_box = gr.Textbox(placeholder="Ask healthcare-related questions (e.g., symptoms, medication info)", container=False, scale=7)

        # Define the chatbot interaction logic
        def user_input(message, history):
            response = generate_response(message, history)
            history.append((message, response))
            return history, ""

        # Configure interface layout
        msg_box.submit(user_input, [msg_box, chatbot], [chatbot, msg_box])

        # Launch Gradio app with custom settings
        app.launch()

# Run the chatbot interface
chat_interface()
